<!-- Projeto desenvolvido por Rafael F. Santos - mfa.rafael@gmail.com-->
# <font color='blue'>Sistema de Recomendação de Ações da B3 utilizando Vetores e Espaço Vetorial</font>

### 1. Definição do Problema:

**Objetivo:**  
Desenvolver um sistema de recomendação de ações da B3 (Bolsa de Valores do Brasil) que, dado um ticker informado, sugira outras ações relacionadas com base na classificação setorial das empresas. A solução deve considerar os seguintes critérios de classificação:  

1. **Setor Econômico**  
2. **Subsetor**  
3. **Segmento**  
4. **Segmento de Listagem da B3**

**Motivação:**  
Investidores frequentemente buscam diversificar suas carteiras com ações de empresas que possuam semelhanças setoriais ou que operem em mercados relacionados. Entretanto, a identificação manual dessas ações pode ser demorada e suscetível a erros. Um sistema de recomendação automatizado pode auxiliar nesse processo, oferecendo sugestões precisas e fundamentadas para facilitar a tomada de decisão.

**Desafio:**  
- Criar um modelo capaz de analisar o ticker fornecido e, com base na classificação setorial, identificar ações correlacionadas.  
- Garantir que o sistema seja capaz de lidar com atualizações periódicas nos dados das empresas listadas na B3.  
- Gerar recomendações úteis e relevantes para diferentes perfis de investidores.

**Produto Esperado:**  
Um sistema de recomendação que forneça uma lista de ações relacionadas ao ticker informado, permitindo aos usuários explorar outras opções dentro do mesmo setor ou setores correlacionados, com transparência nos critérios de recomendação.

### 2. Instalando e Carregando Pacotes

In [1]:
# Instalação/atualização de pacotes, se necessário!

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.
# !pip install -q -U nltk
# !pip install -q -U numpy
# !pip install -q -U pandas
# !pip install -q -U scikit-learn
# !pip install -q -U watermark
# !pip install -q -U openpyxl

In [2]:
# Importação das bibliotecas
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 3. Coleta de Dados

A **classificação setorial da B3** organiza as empresas listadas na bolsa brasileira em categorias hierárquicas, facilitando a análise e comparação entre companhias de setores semelhantes. Essa estrutura é dividida em três níveis:

1. **Setor Econômico**: a categoria mais ampla, agrupando empresas de acordo com a natureza geral de suas atividades.

2. **Subsetor**: uma subdivisão dentro do setor econômico, que refina a classificação com base em características mais específicas das operações das empresas.

3. **Segmento**: a categoria mais específica, detalhando ainda mais as particularidades das atividades empresariais.

Essa classificação auxilia investidores e analistas a compreenderem melhor o mercado, permitindo comparações mais precisas entre empresas de um mesmo segmento ou setor.

Para acessar a classificação setorial completa e atualizada, a B3 disponibiliza um documento oficial em seu site. Você pode baixá-lo diretamente através do seguinte link:

[Classificação Setorial - B3](https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/acoes/consultas/classificacao-setorial/)

Este documento é uma referência valiosa para quem deseja aprofundar-se na estrutura setorial das empresas negociadas na B3 e é a base de dados que será utilizada neste projeto.



In [3]:
# Carregando e compreendendo os dados
df = pd.read_excel('dados/setorial.xlsx')

In [4]:
df.shape

(538, 5)

In [5]:
# Amostragem dos dados
df.head(10)

,CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,NaN
6,NaN,NaN,NaN,CÓDIGO,SEGMENTO
7,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
8,NaN,NaN,BRAVA,BRAV,NM
9,NaN,NaN,COSAN,CSAN,NM


In [6]:
# Amostragem dos dados
df.tail(10)

,CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
528,(M2) Bovespa Mais - Nível 2,NaN,NaN,NaN,NaN
529,(MB) Balcão Organizado Tradicional,NaN,NaN,NaN,NaN
530,NaN,NaN,NaN,NaN,NaN
531,ATENÇÃO,NaN,NaN,NaN,NaN
532,Este trabalho não é uma recomendação de invest...,NaN,NaN,NaN,NaN
533,As informações recebidas das empresas admitida...,NaN,NaN,NaN,NaN
534,nosso site www.b3.com.br.,NaN,NaN,NaN,NaN
535,"Para mais esclarecimentos, sugerimos procurar ...",NaN,NaN,NaN,NaN
536,potenciais das negociações com valores mobiliá...,NaN,NaN,NaN,NaN
537,"B3 S.A. - Brasil, Bolsa, Balcão",NaN,NaN,NaN,NaN


### 4. Tratamento dos dados.

É necessário o tratamento dos dados da planilha, renomeando colunas, e ajustando os dados de maneira que fique mais legível.

In [7]:
# Renomeando as colunas
df.rename(columns={'CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3': 'setor_economico', \
                    'Unnamed: 1': 'subsetor', 'Unnamed: 2': 'segmento', 'Unnamed: 3': 'codigo', \
                    'Unnamed: 4': 'listagem'}, inplace=True)

In [8]:
df.head(10)

,setor_economico,subsetor,segmento,codigo,listagem
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,NaN
6,NaN,NaN,NaN,CÓDIGO,SEGMENTO
7,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
8,NaN,NaN,BRAVA,BRAV,NM
9,NaN,NaN,COSAN,CSAN,NM


In [23]:
# Criando a coluna 'empresa'
df['empresa'] = df['segmento']

In [26]:
df.head(20)

,setor_economico,subsetor,segmento,codigo,listagem,empresa
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,NaN,SEGMENTO
6,NaN,NaN,NaN,CÓDIGO,SEGMENTO,NaN
7,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN,"Exploração, Refino e Distribuição"
8,NaN,NaN,BRAVA,BRAV,NM,BRAVA
9,NaN,NaN,COSAN,CSAN,NM,COSAN


In [28]:
# Reorganizando as colunas
nova_ordem = ['empresa', 'codigo', 'setor_economico', 'subsetor', 'segmento', 'listagem' ]
df = df[nova_ordem]

In [29]:
df.head(20)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,SEGMENTO,LISTAGEM,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,NaN
6,NaN,CÓDIGO,NaN,NaN,NaN,SEGMENTO
7,"Exploração, Refino e Distribuição",NaN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN
8,BRAVA,BRAV,NaN,NaN,BRAVA,NM
9,COSAN,CSAN,NaN,NaN,COSAN,NM


In [30]:
# Excluíndo linhas e resetando o índice
teste = df.drop(index=range(7)).reset_index(drop=True)

In [31]:
teste.head(20)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
0,"Exploração, Refino e Distribuição",NaN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN
1,BRAVA,BRAV,NaN,NaN,BRAVA,NM
2,COSAN,CSAN,NaN,NaN,COSAN,NM
3,PET MANGUINH,RPMG,NaN,NaN,PET MANGUINH,NaN
4,PETROBRAS,PETR,NaN,NaN,PETROBRAS,N2
5,PETRORECSA,RECV,NaN,NaN,PETRORECSA,NM
6,PETRORIO,PRIO,NaN,NaN,PETRORIO,NM
7,RAIZEN,RAIZ,NaN,NaN,RAIZEN,N2
8,ULTRAPAR,UGPA,NaN,NaN,ULTRAPAR,NM
9,VIBRA,VBBR,NaN,NaN,VIBRA,NM


In [34]:
if teste['codigo'].isna:
    teste = teste['segmento'].fillna(method='ffill')

KeyError: 'codigo'

In [33]:
teste.head(25)

0                                   NaN
1                                   NaN
2                                   NaN
3                                   NaN
4                                   NaN
5                              SEGMENTO
6                              SEGMENTO
7     Exploração, Refino e Distribuição
8                                 BRAVA
9                          COSAN       
10                         PET MANGUINH
11                         PETROBRAS   
12                          PETRORECSA 
13                             PETRORIO
14                               RAIZEN
15                         ULTRAPAR    
16                                VIBRA
17              Equipamentos e Serviços
18                         LUPATECH    
19                            OCEANPACT
20                         OSX BRASIL  
21                         OSX BRASIL  
22                             SEGMENTO
23                             SEGMENTO
24                   Minerais Metálicos


In [59]:
# 1- parte da função
def teste(df):
        for coluna in ['setor_economico', 'subsetor', 'segmento']:
                df[coluna] = df.groupby('codigo')[coluna].transform(lambda x: x.ffill().bfill())
        return df

In [ ]:
teste(df).head(25)